In [1]:
import sys
import os
sys.path.append(r"../")
sys.path.append(r"../../")
sys.path.append('/home/wyliu/code/CB-IV')
from utils.imbFun import *
import random
import numpy as np
import argparse
import pandas as pd
import matplotlib.pyplot as plt
from utils import log, CausalDataset

In [2]:
def get_args():
    argparser = argparse.ArgumentParser(description=__doc__)
    # About run setting !!!!
    argparser.add_argument('--seed',default=2021,type=int,help='The random seed')
    argparser.add_argument('--mode',default='vx',type=str,help='The choice of v/x/vx/xx')
    argparser.add_argument('--rewrite_log',default=False,type=bool,help='Whether rewrite log file')
    argparser.add_argument('--use_gpu',default=False,type=bool,help='The use of GPU')
    argparser.add_argument('--ood',default=0,type=float,help='The train dataset of OOD')
    # About data setting ~~~~
    argparser.add_argument('--num',default=10000,type=int,help='The num of train\val\test dataset')
    argparser.add_argument('--num_reps',default=10,type=int,help='The num of train\val\test dataset')
    argparser.add_argument('--ate',default=0,type=float,help='The ate of constant')
    argparser.add_argument('--sc',default=1,type=float,help='The sc')
    argparser.add_argument('--sh',default=0,type=float,help='The sh')
    argparser.add_argument('--one',default=1,type=int,help='The dim of Instrumental variables V')
    argparser.add_argument('--depX',default=0.05,type=float,help='Whether generates harder datasets')
    argparser.add_argument('--depU',default=0.05,type=float,help='Whether generates harder datasets')
    argparser.add_argument('--VX',default=0,type=int,help='The dim of Instrumental variables V')
    argparser.add_argument('--mV',default=2,type=int,help='The dim of Instrumental variables V')
    argparser.add_argument('--mX',default=10,type=int,help='The dim of Confounding variables X')
    argparser.add_argument('--mU',default=4,type=int,help='The dim of Unobserved confounding variables U')
    argparser.add_argument('--mXs',default=2,type=int,help='The dim of Noise variables X')
    
    argparser.add_argument('--storage_path',default='../../Data/',type=str,help='The dir of data storage')
    # Syn
    argparser.add_argument('--syn_alpha',default=0.01,type=float,help='')
    argparser.add_argument('--syn_lambda',default=0.0001,type=float,help='')
    argparser.add_argument('--syn_twoStage',default=True,type=bool,help='')
    # About Debug or Show
    argparser.add_argument('--verbose',default=1,type=int,help='The level of verbose')
    argparser.add_argument('--epoch_show',default=5,type=int,help='The epochs of show time')
    args = argparser.parse_args(args=[])
    return args

args = get_args()

def get_FLAGS():
    ''' Define parameter flags '''
    FLAGS = tf.app.flags.FLAGS

    tf.app.flags.DEFINE_integer('lrate_decay_num', 100, """NUM_ITERATIONS_PER_DECAY. """)
    tf.app.flags.DEFINE_integer('seed', 2021, """Seed. """)
    tf.app.flags.DEFINE_integer('debug', 0, """Debug mode. """)
    tf.app.flags.DEFINE_integer('save_rep', 0, """Save representations after training. """)
    tf.app.flags.DEFINE_integer('output_csv',0,"""Whether to save a CSV file with the results""")
    tf.app.flags.DEFINE_integer('output_delay', 100, """Number of iterations between log/loss outputs. """)
    tf.app.flags.DEFINE_string('x_key', 'x', """Which key to use (x/xu/vxu)""")
    tf.app.flags.DEFINE_string('loss', 'l2', """Which loss function to use (l1/l2/log)""")
    tf.app.flags.DEFINE_integer('n_in', 3, """Number of representation layers. """)
    tf.app.flags.DEFINE_integer('n_out', 5, """Number of regression layers. """)
    tf.app.flags.DEFINE_float('p_alpha', 1, """Imbalance regularization param. """)
    tf.app.flags.DEFINE_float('p_lambda', 1e-4, """Weight decay regularization parameter. """)
    tf.app.flags.DEFINE_integer('rep_weight_decay', 0, """Whether to penalize representation layers with weight decay""")
    tf.app.flags.DEFINE_float('dropout_in', 1.0, """Input layers dropout keep rate. """)
    tf.app.flags.DEFINE_float('dropout_out', 1.0, """Output layers dropout keep rate. """)
    tf.app.flags.DEFINE_string('nonlin', 'elu', """Kind of non-linearity. Default relu. """)
    tf.app.flags.DEFINE_float('lrate', 5e-4, """Learning rate. """)
    tf.app.flags.DEFINE_float('decay', 0.3, """RMSProp decay. """)
    tf.app.flags.DEFINE_integer('batch_size', 256, """Batch size. """)
    tf.app.flags.DEFINE_integer('dim_in', 256, """Pre-representation layer dimensions. """)
    tf.app.flags.DEFINE_integer('dim_out', 256, """Post-representation layer dimensions. """)
    tf.app.flags.DEFINE_integer('batch_norm', 0, """Whether to use batch normalization. """)
    tf.app.flags.DEFINE_string('normalization', 'none', """How to normalize representation (after batch norm). none/bn_fixed/divide/project """)
    tf.app.flags.DEFINE_float('rbf_sigma', 0.1, """RBF MMD sigma """)
    tf.app.flags.DEFINE_integer('experiments', 2, """Number of experiments. """)
    tf.app.flags.DEFINE_integer('iterations', 300, """Number of iterations. """)
    tf.app.flags.DEFINE_float('weight_init', 0.1, """Weight initialization scale. """)
    tf.app.flags.DEFINE_float('lrate_decay', 0.97, """Decay of learning rate every 100 iterations """)
    tf.app.flags.DEFINE_integer('wass_iterations', 10, """Number of iterations in Wasserstein computation. """)
    tf.app.flags.DEFINE_float('wass_lambda', 10.0, """Wasserstein lambda. """)
    tf.app.flags.DEFINE_integer('wass_bpt', 1, """Backprop through T matrix? """)
    tf.app.flags.DEFINE_integer('varsel', 0, """Whether the first layer performs variable selection. """)
    tf.app.flags.DEFINE_string('outdir', '../Data/DRCFR/results/', """Output directory. """)
    tf.app.flags.DEFINE_string('datadir', '../Data/DRCFR/data/Syn_1.0_1.0_0/2_4_4/', """Data directory. """)
    tf.app.flags.DEFINE_string('dataform', 'train_0.csv', """Training data filename form. """)
    tf.app.flags.DEFINE_string('data_val', 'val_0.csv', """Valid data filename form. """)
    tf.app.flags.DEFINE_string('data_test', 'test_0.csv', """Test data filename form. """)
    tf.app.flags.DEFINE_integer('sparse', 0, """Whether data is stored in sparse format (.x, .y). """)
    tf.app.flags.DEFINE_integer('repetitions', 1, """Repetitions with different seed.""")
    tf.app.flags.DEFINE_integer('use_p_correction', 0, """Whether to use population size p(t) in mmd/disc/wass.""")
    tf.app.flags.DEFINE_string('optimizer', 'Adam', """Which optimizer to use. (RMSProp/Adagrad/GradientDescent/Adam)""")
    tf.app.flags.DEFINE_string('imb_fun', 'wass', """Which imbalance penalty to use (mmd_lin/mmd_rbf/mmd2_lin/mmd2_rbf/lindisc/wass). """)
    tf.app.flags.DEFINE_integer('pred_output_delay', 200, """Number of iterations between prediction outputs. (-1 gives no intermediate output). """)
    tf.app.flags.DEFINE_float('val_part', 0.3, """Validation part. """)
    tf.app.flags.DEFINE_boolean('split_output', 1, """Whether to split output layers between treated and control. """)
    tf.app.flags.DEFINE_boolean('reweight_sample', 1, """Whether to reweight sample for prediction loss with average treatment probability. """)
    tf.app.flags.DEFINE_boolean('twoStage', 1, """twoStage. """)
    tf.app.flags.DEFINE_string('f', '', 'kernel')
    tf.app.flags.DEFINE_string('ip', '', 'kernel')
    tf.app.flags.DEFINE_integer('mV', 2, """The dim of Instrumental variables V.""")
    tf.app.flags.DEFINE_integer('mX', 4, """The dim of Confounding variables X.""")
    tf.app.flags.DEFINE_integer('mU', 4, """The dim of Unobserved confounding variables U.""")
    tf.app.flags.DEFINE_float('ood', 0., """ood. """)

    if FLAGS.sparse:
        import scipy.sparse as sparse

    return FLAGS

In [3]:
FLAGS = get_FLAGS()
# set OOD path
''' bias rate '''
args.ood = -3.0
br = [-3.0, -2.5, -2.0, -1.5, -1.3, 0.0, 1.3, 1.5, 2.0, 2.5, 3.0]
brdc = {-3.0: 'n30', -2.5:'n25', -2.0:'n20', -1.5:'n15', -1.3:'n13', 1.3:'p13', 1.5:'p15', 2.0:'p20', 2.5:'p25', 3.0:'p30', 0.0:'0'}
which_benchmark = 'SynOOD_'+'_'.join(str(item) for item in [args.sc, args.sh, args.one, args.depX, args.depU,args.VX])
which_dataset = '_'.join(str(item) for item in [args.mV, args.mX, args.mU, args.mXs])
resultDir = args.storage_path + f'/results/{which_benchmark}_{which_dataset}_{args.mode}/ood{brdc[args.ood]}/'
dataDir = f'{args.storage_path}/data/{which_benchmark}/{which_dataset}/'
os.makedirs(os.path.dirname(resultDir), exist_ok=True)
logfile = f'{resultDir}/log.txt'
''' bias rate '''
br = [-3.0, -2.5, -2.0, -1.5, -1.3, 0.0, 1.3, 1.5, 2.0, 2.5, 3.0]
brdc = {-3.0: 'n30', -2.5:'n25', -2.0:'n20', -1.5:'n15', -1.3:'n13', 1.3:'p13', 1.5:'p15', 2.0:'p20', 2.5:'p25', 3.0:'p30', 0.0:'0'}


In [4]:
def mmd2_rbf_ood(Xc,Xt,p,sig):
    """ Computes the l2-RBF MMD for X given t """

    # it = tf.where(t>0)[:,0]
    # ic = tf.where(t<1)[:,0]

    # Xc = tf.gather(X,ic)
    # Xt = tf.gather(X,it)
    Xc = tf.convert_to_tensor(Xc, dtype=tf.float32)
    Xt = tf.convert_to_tensor(Xt, dtype=tf.float32)
    Kcc = tf.exp(-pdist2sq(Xc,Xc)/tf.square(sig))
    Kct = tf.exp(-pdist2sq(Xc,Xt)/tf.square(sig))
    Ktt = tf.exp(-pdist2sq(Xt,Xt)/tf.square(sig))

    m = tf.to_float(tf.shape(Xc)[0])
    n = tf.to_float(tf.shape(Xt)[0])

    mmd = tf.square(1.0-p)/(m*(m-1.0))*(tf.reduce_sum(Kcc)-m)
    mmd = mmd + tf.square(p)/(n*(n-1.0))*(tf.reduce_sum(Ktt)-n)
    mmd = mmd - 2.0*p*(1.0-p)/(m*n)*tf.reduce_sum(Kct)
    mmd = 4.0*mmd

    return mmd

In [41]:
def wasserstein_ood(Xc,Xt, p,lam=10,its=10,sq=False,backpropT=False):
    """ Returns the Wasserstein distance between treatment groups """

    # it = tf.where(t1>0)[:,0]
    # ic = tf.where(t2>0)[:,0]
    # Xc = tf.gather(X1,ic)
    # Xt = tf.gather(X2,it)
    Xc = tf.convert_to_tensor(Xc, dtype=tf.float32)
    Xt = tf.convert_to_tensor(Xt, dtype=tf.float32)
    nc = tf.to_float(tf.shape(Xc)[0])
    nt = tf.to_float(tf.shape(Xt)[0])

    ''' Compute distance matrix'''
    if sq:
        M = pdist2sq(Xt,Xc)
    else:
        M = safe_sqrt(pdist2sq(Xt,Xc))

    ''' Estimate lambda and delta '''
    M_mean = tf.reduce_mean(M)
    M_drop = tf.nn.dropout(M,10/(nc*nt))
    delta = tf.stop_gradient(tf.reduce_max(M))
    eff_lam = tf.stop_gradient(lam/M_mean)

    ''' Compute new distance matrix '''
    Mt = M
    row = delta*tf.ones(tf.shape(M[0:1,:]))
    col = tf.concat([delta*tf.ones(tf.shape(M[:,0:1])),tf.zeros((1,1))],0)
    Mt = tf.concat([M,row],0)
    Mt = tf.concat([Mt,col],1)

    ''' Compute marginal vectors '''
    # print('nt', nt)
    a = tf.concat([p*tf.ones((10000,1))/nt, (1-p)*tf.ones((1,1))],0)
    b = tf.concat([(1-p)*tf.ones((10000,1))/nc, p*tf.ones((1,1))],0)

    ''' Compute kernel matrix'''
    Mlam = eff_lam*Mt
    K = tf.exp(-Mlam) + 1e-6 # added constant to avoid nan
    U = K*Mt
    ainvK = K/a

    u = a
    for i in range(0,its):
        u = 1.0/(tf.matmul(ainvK,(b/tf.transpose(tf.matmul(tf.transpose(u),K)))))
    v = b/(tf.transpose(tf.matmul(tf.transpose(u),K)))

    T = u*(tf.transpose(v)*K)

    if not backpropT:
        T = tf.stop_gradient(T)

    E = T*Mt
    D = 2*tf.reduce_sum(E)

    return D, Mlam

In [66]:
exp = 0
args.ood = 3.0
train_df = pd.read_csv(dataDir + f'{exp}/{args.mode}/ood_{brdc[args.ood]}/train.csv')
val_df = pd.read_csv(dataDir + f'{exp}/{args.mode}/ood_{brdc[args.ood]}/val.csv')
test_df = pd.read_csv(dataDir + f'{exp}/{args.mode}/ood_{brdc[args.ood]}/test.csv')
train = CausalDataset(train_df, variables = ['u','x','v','xs','z','p','s','m','t','g','y','f','c'], observe_vars=['v','x','xs'])
val = CausalDataset(val_df, variables = ['u','x','v','xs','z','p','s','m','t','g','y','f','c'], observe_vars=['v','x','xs'])
test = CausalDataset(test_df, variables = ['u','x','v','xs','z','p','s','m','t','g','y','f','c'], observe_vars=['v','x','xs'])

x_list = [np.concatenate((train.x, train.xs), 1), 
            np.concatenate((val.x, val.xs), 1), 
            np.concatenate((test.x, test.xs), 1)]

train = {'x':x_list[0],
        't':train.t,
        's':train.s,
        'g':train.g,
        'yf':train.y,
        'ycf':train.f}
val = {'x':x_list[1],
        't':val.t,
        's':val.s,
        'g':val.g,
        'yf':val.y,
        'ycf':val.f}
test = {'x':x_list[2],
        't':test.t,
        's':test.s,
        'g':test.g,
        'yf':test.y,
        'ycf':test.f}

In [67]:
wass_dist = []
mmd_dist = []
results_ood = [wass_dist, mmd_dist]
name_ood = ["wass_dist", "mmd_dist"]
for exp in range(10): 
    l1 = []
    l2 = []
    for r in br:    
        train_df_ood = pd.read_csv(dataDir + f'{exp}/{args.mode}/ood_{brdc[r]}/train.csv')
        # val_df_ood = pd.read_csv(dataDir + f'{exp}/{args.mode}/ood_{brdc[args.ood]}/val.csv')
        # test_df_ood = pd.read_csv(dataDir + f'{exp}/{args.mode}/ood_{brdc[args.ood]}/test.csv')
        train_ood = CausalDataset(train_df_ood, variables = ['u','x','v','xs','z','p','s','m','t','g','y','f','c'], observe_vars=['v','x','xs'])
        # val_ood = CausalDataset(val_df_ood, variables = ['u','x','v','xs','z','p','s','m','t','g','y','f','c'], observe_vars=['v','x','xs'])
        # test_ood = CausalDataset(test_df_ood, variables = ['u','x','v','xs','z','p','s','m','t','g','y','f','c'], observe_vars=['v','x','xs'])
        print(dataDir + f'{exp}/{args.mode}/ood_{brdc[r]}/train.csv')
        x_list = [np.concatenate((train_ood.x, train_ood.xs), 1)]

        train_ood = {'x':x_list[0],
                't':train_ood.t,
                's':train_ood.s,
                'g':train_ood.g,
                'yf':train_ood.y,
                'ycf':train_ood.f}
        p_ipm = 0.5
        imb_dist, imB_mat = wasserstein_ood(train['x'],train_ood['x'],p_ipm)
        # 创建 TensorFlow 会话
        with tf.Session() as sess:
            # 执行计算图并获取张量的值
            tensor_value = sess.run(imb_dist)
        print(tensor_value)
        l1.append(tensor_value)

        imb_dist_mmd = mmd2_rbf_ood(train['x'],train_ood['x'],p_ipm, FLAGS.rbf_sigma)
        # 创建 TensorFlow 会话
        with tf.Session() as sess:
            # 执行计算图并获取张量的值
            tensor_value_mmd = sess.run(imb_dist_mmd)
        print(tensor_value_mmd)
        l2.append(tensor_value_mmd)
    wass_dist.append(l1)
    mmd_dist.append(l2)


for res, name in zip(results_ood, name_ood):
    res.append(np.mean(res[:][:args.num_reps],0))
    res.append(np.std(res[:][:args.num_reps],0))
    res_df = pd.DataFrame(np.array(res), columns=[brdc[r] for r in br ]).round(4)
    res_df.to_csv(resultDir + f'IPM_{args.mode}_{brdc[r]}_' + name + '.csv', index=False)

../../Data//data/SynOOD_1_0_1_0.05_0.05_0/2_10_4_2/0/vx/ood_n30/train.csv
3.781878
1
2
-1.2800798e-06
../../Data//data/SynOOD_1_0_1_0.05_0.05_0/2_10_4_2/0/vx/ood_n25/train.csv
3.7763948
1
2
-1.4200971e-06
../../Data//data/SynOOD_1_0_1_0.05_0.05_0/2_10_4_2/0/vx/ood_n20/train.csv
3.7748523
1
2
-1.3400266e-06
../../Data//data/SynOOD_1_0_1_0.05_0.05_0/2_10_4_2/0/vx/ood_n15/train.csv
3.7511878
1
2
-1.4199364e-06
../../Data//data/SynOOD_1_0_1_0.05_0.05_0/2_10_4_2/0/vx/ood_n13/train.csv
3.7230735
1
2
-1.3398154e-06
../../Data//data/SynOOD_1_0_1_0.05_0.05_0/2_10_4_2/0/vx/ood_0/train.csv
3.6469154
1
2
-2.0401626e-06
../../Data//data/SynOOD_1_0_1_0.05_0.05_0/2_10_4_2/0/vx/ood_p13/train.csv
2.8291416
1
2
-2.9639996e-05
../../Data//data/SynOOD_1_0_1_0.05_0.05_0/2_10_4_2/0/vx/ood_p15/train.csv
2.695561
1
2
-4.781998e-05
../../Data//data/SynOOD_1_0_1_0.05_0.05_0/2_10_4_2/0/vx/ood_p20/train.csv
2.5198214
1
2
-7.604012e-05
../../Data//data/SynOOD_1_0_1_0.05_0.05_0/2_10_4_2/0/vx/ood_p25/train.csv
2.438

In [50]:
resultDir

'../../Data//results/SynOOD_1_0_1_0.05_0.05_0_2_10_4_2_vx/oodn30/'

In [44]:
p_ipm = 0.5
imb_dist, imB_mat = wasserstein_ood(train['x'],test['x'],p_ipm)

In [45]:
import tensorflow as tf

# 假设有一个名为 tensor 的 TensorFlow 张量
tensor_v = imb_dist
# 创建 TensorFlow 会话
with tf.Session() as sess:
    # 执行计算图并获取张量的值
    tensor_value = sess.run(tensor_v)

# 打印张量的值
print(tensor_value)

2.702166


In [16]:
X = train['x']
t = train['t']
lam=10
its=10
sq=False
backpropT=False
p = 0.5

In [10]:
X = tf.convert_to_tensor(X, dtype=tf.float32)
t = tf.convert_to_tensor(t, dtype=tf.float32)
it = tf.where(t>0)[:,0]
ic = tf.where(t<1)[:,0]
Xc = tf.gather(X,ic)
Xt = tf.gather(X,it)
nc = tf.to_float(tf.shape(Xc)[0])
nt = tf.to_float(tf.shape(Xt)[0])

In [14]:
''' Compute distance matrix'''
if sq:
    M = pdist2sq(Xt,Xc)
else:
    M = safe_sqrt(pdist2sq(Xt,Xc))

''' Estimate lambda and delta '''
M_mean = tf.reduce_mean(M)
M_drop = tf.nn.dropout(M,10/(nc*nt))
delta = tf.stop_gradient(tf.reduce_max(M))
eff_lam = tf.stop_gradient(lam/M_mean)

''' Compute new distance matrix '''
Mt = M
row = delta*tf.ones(tf.shape(M[0:1,:]))
col = tf.concat([delta*tf.ones(tf.shape(M[:,0:1])),tf.zeros((1,1))],0)
Mt = tf.concat([M,row],0)
Mt = tf.concat([Mt,col],1)

In [31]:
''' Compute marginal vectors '''
a = tf.concat([p*tf.ones((10000,1))/nt, (1-p)*tf.ones((1,1))],0)
b = tf.concat([(1-p)*tf.ones((10000,1))/nc, p*tf.ones((1,1))],0)

In [34]:
''' Compute marginal vectors '''
a = tf.concat([p*tf.ones(tf.shape(tf.where(t>0)[:,0:1]))/nt, (1-p)*tf.ones((1,1))],0)
b = tf.concat([(1-p)*tf.ones(tf.shape(tf.where(t<1)[:,0:1]))/nc, p*tf.ones((1,1))],0)

In [35]:
import tensorflow as tf

# 假设有一个名为 tensor 的 TensorFlow 张量
tensor_v = a
# 创建 TensorFlow 会话
with tf.Session() as sess:
    # 执行计算图并获取张量的值
    tensor_value = sess.run(tensor_v)

# 打印张量的值
print(tensor_value.shape)

(5384, 1)
